In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import nltk #natural language tool kit
import string

In [ ]:

banks=pd.read_excel("/content/BankReviews.xlsx")

In [ ]:
banks.head()

,Date,Stars,Reviews,BankName
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage


In [ ]:
banks.shape

(505, 4)

In [ ]:
banks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      505 non-null    datetime64[ns]
 1   Stars     505 non-null    int64         
 2   Reviews   505 non-null    object        
 3   BankName  505 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 15.9+ KB


In [ ]:
banks=banks.drop(['Date','BankName'], axis=1)

In [ ]:
def text_lowercase(text):
    return text.lower()
banks['Reviews_clean']=banks['Reviews'].apply(text_lowercase)

In [ ]:
import string
string.punctuation
def remove_punct(text):
    text_nonpunc="".join([char for char in text if char not in string.punctuation])
    return text_nonpunc

banks['Reviews_clean']=banks['Reviews_clean'].apply(remove_punct)

In [ ]:
banks.head()


,Stars,Reviews,Reviews_clean
0,5,"Great job, Wyndham Capital! Each person was pr...",great job wyndham capital each person was prof...
1,5,Matthew Richardson is professional and helpful...,matthew richardson is professional and helpful...
2,5,We had a past experience with Wyndham Mortgage...,we had a past experience with wyndham mortgage...
3,5,We have been dealing with Brad Thomka from the...,we have been dealing with brad thomka from the...
4,5,I can't express how grateful I am for the supp...,i cant express how grateful i am for the suppo...


In [ ]:
def remove_whitespaces(text):
    return " ".join(text.split())

In [ ]:
banks['Reviews_clean']=banks['Reviews_clean'].apply(lambda x:remove_whitespaces(x))

In [ ]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

banks['Reviews_clean'] = banks['Reviews_clean'].apply(lambda x: tokenize(x))

banks.head()

,Stars,Reviews,Reviews_clean
0,5,"Great job, Wyndham Capital! Each person was pr...","[great, job, wyndham, capital, each, person, w..."
1,5,Matthew Richardson is professional and helpful...,"[matthew, richardson, is, professional, and, h..."
2,5,We had a past experience with Wyndham Mortgage...,"[we, had, a, past, experience, with, wyndham, ..."
3,5,We have been dealing with Brad Thomka from the...,"[we, have, been, dealing, with, brad, thomka, ..."
4,5,I can't express how grateful I am for the supp...,"[i, cant, express, how, grateful, i, am, for, ..."


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(tokenized_list):
    filtered_text=[word for word in tokenized_list if word not in stop]
    return filtered_text

In [ ]:
banks['Reviews_clean']=banks['Reviews_clean'].apply(lambda x:remove_stopwords(x))

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
wn=nltk.stem.WordNetLemmatizer()
def lemmatizing(text):
    lemma=[wn.lemmatize(word) for word in text]
    return lemma

In [ ]:
word_count={}
for sentence in banks['Reviews_clean']:
    for word in sentence:
        if word not in word_count:
            word_count[word]=1
        else:
            word_count[word]+=1

In [ ]:
df=pd.DataFrame(word_count.items(), columns=['word','word_count'])
df=df.sort_values('word_count', ascending=False).reset_index(drop=True)
df.head()

,word,word_count
0,x000d,715
1,loan,311
2,would,246
3,process,243
4,us,207


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
problems = banks.loc[banks['Stars'] < 3]['Reviews']
problems.head()

,Reviews
17,This was the worst experience ever. It was lik...
18,"A good rate but a very frustrating process, co..."
19,They were not upfront. Learn from my mistake.....
20,"Initially, the Mortgage Broker was very friend..."
21,"Initially, the Mortgage Broker was very friend..."


In [ ]:
vectorizer=TfidfVectorizer(stop_words='english')
X=vectorizer.fit_transform(problems)

In [ ]:
vectorizer=TfidfVectorizer(min_df=0.05, max_df=0.8, max_features=2000)
X = vectorizer.fit_transform(banks.Reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.28473799,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.2201384 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.17588462],
       ...,
       [0.        , 0.09409383, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.13076688,
        0.17352575],
       [0.        , 0.05152938, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X, banks.Stars, test_size=0.3)

In [ ]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(X_train,y_train)

LogisticRegression()

In [ ]:
log.score(X_train,y_train)

0.9093484419263456

In [ ]:
log.score(X_test,y_test)

0.8881578947368421

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(max_depth=3,min_samples_split=7,min_samples_leaf=3)
dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=3, min_samples_split=7)

In [ ]:
dt.score(X_train,y_train)

0.9291784702549575

In [ ]:
dt.score(X_test,y_test)

0.868421052631579

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier